<p style="align: center;"><img align=center src="https://drive.google.com/uc?export=view&id=1I8kDikouqpH4hf7JBiSYAeNT2IO52T-T" width=600 height=480/></p>
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Generative adversarial networks</b></h3>



В этом домашнем задании вы обучите GAN генерировать лица людей и посмотрите на то, как можно оценивать качество генерации

In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import clear_output
%matplotlib inline

#sns.set(style='darkgrid', font_scale=1.2)

## Часть 1. Подготовка данных (1 балл)

В качестве обучающей выборки возьмем часть датасета [Flickr Faces](https://github.com/NVlabs/ffhq-dataset), который содержит изображения лиц людей в высоком разрешении (1024х1024). Оригинальный датасет очень большой, поэтому мы возьмем его часть. Скачать датасет можно [здесь](https://drive.google.com/file/d/1KWPc4Pa7u2TWekUvNu9rTSO0U2eOlZA9/view?usp=sharing)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#cd /content/drive/MyDrive/Colab Notebooks/ML/DLS

In [ ]:
#!ls

In [2]:
DATA_DIR = '../input/facesdatasetsmall'

Давайте загрузим наши изображения. Напишите функцию, которая строит DataLoader для изображений, при этом меняя их размер до нужного значения (размер 1024 слишком большой, поэтому мы рекомендуем взять размер 128 либо немного больше)

In [3]:
def get_dataloader(image_size, batch_size):
  """
  Builds dataloader for training data.
  Use tt.Compose and tt.Resize for transformations
  :param image_size: height and wdith of the image
  :param batch_size: batch_size of the dataloader
  :returns: DataLoader object 
  """
  # TODO: resize images, convert them to tensors and build dataloader
  data_ds = ImageFolder(DATA_DIR, transform = tt.Compose([
      tt.Resize(image_size),
      tt.ToTensor()
  ]))

  loader = DataLoader(data_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
  return loader

In [4]:
image_size = 128
batch_size = 64

#TODO: build dataloader and transfer it to device
train_dl = get_dataloader(image_size, batch_size)

In [5]:
batch = next(iter(train_dl))[0]
print(batch.size())

In [7]:
len(train_dl.dataset)

In [11]:
# посмотрим на наши изображения
n_pic = 10

plt.figure(figsize=(18, 8))
for i in range(n_pic):
    plt.subplot(2, n_pic // 2, i + 1)
    plt.imshow(batch[i].permute(1, 2, 0))
    plt.axis('off')
plt.suptitle("Some examples from dataset")
plt.show()

## Часть 2. Построение и обучение модели (2 балла)

Сконструируйте генератор и дискриминатор. Помните, что:
* дискриминатор принимает на вход изображение (тензор размера `3 x image_size x image_size`) и выдает вероятность того, что изображение настоящее (тензор размера 1)

* генератор принимает на вход тензор шумов размера `latent_size x 1 x 1` и генерирует изображение размера `3 x image_size x image_size`

In [43]:
discriminator = nn.Sequential(
    # in: 3 x 128 x 128
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.MaxPool2d(kernel_size=2),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.MaxPool2d(kernel_size=2),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 8 x 8

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.MaxPool2d(kernel_size=2),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 2 x 2
    
    nn.Conv2d(256, 1, kernel_size=2, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten(),
    nn.Sigmoid()
    )   

In [44]:
latent_size = 64

generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 1024, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(1024),
    nn.ReLU(True),
    # out: 1024 x 4 x 4

    nn.ConvTranspose2d(1024, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 8 x 8

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 16 x 16

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 32 x 32

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 64 x 64

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Sigmoid()
    # out: 3 x 128 x 128
)

Перейдем теперь к обучению нашего GANа. Алгоритм обучения следующий:
1. Учим дискриминатор:
  * берем реальные изображения и присваиваем им метку 1
  * генерируем изображения генератором и присваиваем им метку 0
  * обучаем классификатор на два класса

2. Учим генератор:
  * генерируем изображения генератором и присваиваем им метку 0
  * предсказываем дискриминаторором, реальное это изображение или нет


В качестве функции потерь берем бинарную кросс-энтропию

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [118]:
def fit(model, criterion, n_epochs, lr, dataloader, buffer_size):
    # buffer size is a number of batches that can be included in buffer
    # TODO: build optimizers and train your GAN
    gen_losses = []
    disc_losses = []
    real_scores = []
    fake_scores = []

    model["discriminator"].train()
    model["generator"].train()
    torch.cuda.empty_cache()
    
    buffer_flag = 0
    buffer = []

    optimizer = {
        "discriminator": torch.optim.Adam(model["discriminator"].parameters(), lr=lr),
        "generator": torch.optim.Adam(model["generator"].parameters(), lr=lr)
    }

    for epoch in tqdm(range(n_epochs)):
        gen_losses_per_epoch = []
        disc_losses_per_epoch = []
        real_scores_per_epoch = []
        fake_scores_per_epoch = []
        
        if epoch == 0:
            buffer_flag = 1
        
        for real_images, _ in tqdm(dataloader):
            
            real_images = real_images.to(device)
            shape = real_images.size()
            
            # Train discriminator
            optimizer["discriminator"].zero_grad()

            # Pass real images through discriminator
            real_preds = model["discriminator"](real_images)
            #noise = torch.FloatTensor(shape[0], 1).uniform_(0, 0.05).to(device)
            real_labels = torch.ones(shape[0], 1, device=device)
            real_loss = criterion["discriminator"](real_preds, real_labels)
            cur_real_score = torch.mean(real_preds).item()

            # Create buffer
            if buffer_flag == 1:
                for i in range(buffer_size):
                    latent = torch.randn(shape[0], latent_size, 1, 1, device=device)
                    fake_images = model["generator"](latent)
                    buffer.append(fake_images)
                buffer_flag = 0
                
            # Generate images
            latent = torch.randn(shape[0], latent_size, 1, 1, device=device)
            fake_images_cur1 = buffer.pop(0)
            fake_images1 = model["generator"](latent)
            buffer.append(fake_images1)
            
            # Pass fake images through discriminator
            fake_labels = torch.zeros(shape[0], 1, device=device)
            fake_preds = model["discriminator"](fake_images_cur1)
            fake_loss = criterion["discriminator"](fake_preds, fake_labels)
            cur_fake_score = torch.mean(fake_preds).item()
            
            real_scores_per_epoch.append(cur_real_score)
            fake_scores_per_epoch.append(cur_fake_score)
            
            # Update discriminator loss
            disc_loss = real_loss + fake_loss
            disc_loss.backward()
            optimizer["discriminator"].step()
            disc_losses_per_epoch.append(disc_loss.item())
        
            # Train generator
            optimizer["generator"].zero_grad()

            # Generate fake images
            latent = torch.randn(shape[0], latent_size, 1, 1, device=device)
            fake_images_cur2 = buffer[0]
            fake_images2 = model["generator"](latent)
            buffer.append(fake_images2)

            # Try to fool discriminator
            preds = model["discriminator"](fake_images_cur2)
            labels = torch.ones(shape[0], 1, device=device)
            gen_loss = criterion["generator"](preds, labels)

            # Update generators weights
            gen_loss.backward()
            optimizer["generator"].step()
            gen_losses_per_epoch.append(gen_loss.item())

        #Record losses and scores
        gen_losses.append(np.mean(gen_losses_per_epoch))
        disc_losses.append(np.mean(disc_losses_per_epoch))
        real_scores.append(np.mean(real_scores_per_epoch))
        fake_scores.append(np.mean(fake_scores_per_epoch))

        # show intermediate results
        clear_output(wait=True)
        fig = plt.figure(figsize=(24, 8))
        for i in range(6):
            ax = fig.add_subplot(2, 6, i + 1)
            ax.imshow(fake_images[i].permute(1, 2, 0).detach().cpu().numpy())
            ax.set_title("Fake image")
            ax.axis('off')

        #draw losses and scores
        if(epoch != 0):
            ax1 = fig.add_subplot(2, 2, 3)
            epochs = np.arange(1, epoch + 2)
            ax1.plot(epochs, gen_losses, label='generator')
            ax1.plot(epochs, disc_losses, label='discriminator')
            ax1.set_title("BCE loss")
            ax1.set_xlabel('epochs')
            ax1.set_ylabel('loss')
            ax1.grid()
            ax1.legend(fontsize=14)
            
            ax2 = fig.add_subplot(2, 2, 4)
            epochs = np.arange(1, epoch + 2)
            ax2.plot(epochs, real_scores, label='real')
            ax2.plot(epochs, fake_scores, label='fake')
            ax2.set_title("Scores")
            ax2.set_xlabel('epochs')
            ax2.set_ylabel('score')
            ax2.grid()
            ax2.legend(fontsize=14)

        plt.suptitle(f"{epoch+1} / {n_epochs} - generator_loss: {gen_losses[-1]:0.4f}, discriminator: {disc_losses[-1]:0.4f}")
        plt.show()

        #if (epoch + 1 == epochs):
        #    torch.save(model.state_dict(), 'gan')

    return gen_losses, disc_losses, real_scores, fake_scores

In [119]:
lr = 0.0003

model = {
    "discriminator": discriminator.to(device),
    "generator": generator.to(device)
}

criterion_bce = {
    "discriminator": nn.BCELoss(),
    "generator": nn.BCELoss()
}

criterion_mse = {
    "discriminator": nn.MSELoss(),
    "generator": nn.MSELoss()
}

In [120]:
%%time
epochs = 30
gen_losses, disc_losses, real_scores, fake_scores = fit(model, criterion_mse, epochs, lr, train_dl, 5)

Постройте графики лосса для генератора и дискриминатора. Что вы можете сказать про эти графики?

## Часть 3. Генерация изображений (1 балл)

Теперь давайте оценим качество получившихся изображений. Напишите функцию, которая выводит изображения, сгенерированные нашим генератором

In [31]:
n_images = 10
fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)
fake_images = model["generator"](fixed_latent)

In [41]:
def show_images(generated, n_images):
  # TODO: show generated images
    fig = plt.figure(figsize=(24, 8))
    for i in range(n_images):
        ax = fig.add_subplot(2, 5, i + 1)
        ax.imshow(fake_images[i].permute(1, 2, 0).detach().cpu().numpy())
        ax.axis('off')

In [42]:
show_images(fake_images, n_images)

Как вам качество получившихся изображений?

## Часть 4. Leave-one-out-1-NN classifier accuracy (6 баллов)

### 4.1. Подсчет accuracy (4 балла)

Не всегда бывает удобно оценивать качество сгенерированных картинок глазами. В качестве альтернативы вам предлагается реализовать следующий подход:
  * Сгенерировать столько же фейковых изображений, сколько есть настоящих в обучающей выборке. Присвоить фейковым метку класса 0, настоящим – 1.
  * Построить leave-one-out оценку: обучить 1NN Classifier (`sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)`) предсказывать класс на всех объектах, кроме одного, проверить качество (accuracy) на оставшемся объекте. В этом вам поможет `sklearn.model_selection.LeaveOneOut`

Что вы можете сказать о получившемся результате? Какой accuracy мы хотели бы получить и почему?

### 4.2. Визуализация распределений (2 балла)

Давайте посмотрим на то, насколько похожи распределения настоящих и фейковых изображений. Для этого воспользуйтесь методом, снижающим размерность (к примеру, TSNE) и изобразите на графике разным цветом точки, соответствующие реальным и сгенерированным изображенияи

Прокомментируйте получившийся результат: